In [1]:
import pdftotext
import pandas as pd
import numpy as np
import re
import os

### Define function for "Cleaning" and "participants list"

In [47]:
def cleaning_text(contents):
    ### Cleaning all the unwanted rows in the transcript
    df = pd.DataFrame(contents)

    # remove the unnessary string
    df[0] = df[0].str.replace('\n','')
    df[0] = df[0].str.replace('Bloomberg Transcript','')
    df[0] = df[0].str.replace('\x0c\n','')
    df[0] = df[0].str.replace('FINAL','')
    df[0] = df[0].str.replace('A - ','')
    df[0] = df[0].str.replace('Q - ','')

    # using re to remove the unnessary string
    def drop_unnessary(x):
        page = re.findall(r'Page \d+ of \d+', x) # 'page ... of ... '
        BIO = re.findall(r'{BIO', x) # '{BIO 18731996 <GO>}'
        Company_Name = re.findall(r'Company N ame:', x) # 'Company N ame: H annover Rueck SE'
        Company_Ticker = re.findall(r'Company Ticker:', x) # 'Company Ticker: H N R1 GR Equity'
        Date = re.findall(r'Date:', x) # Date: 2015-03-10
        if page == [] and BIO == [] and Company_Name == [] and Company_Ticker == [] and Date == []:
            return True
        else:
            return False

    true_false = df[0].apply(lambda x: drop_unnessary(x))
    df = df[true_false]

    # drop the final page declaration
    df = df[df[0] != 'This transcript may not be 100 percent accurate and may contain misspellings and other']
    df = df[df[0] != 'inaccuracies. This transcript is provided "as is", without express or implied warranties of']
    df = df[df[0] != 'any kind. Bloomberg retains all rights to this transcript and provides it solely for your']
    df = df[df[0] != 'personal, non-commercial use. Bloomberg, its suppliers and third-party agents shall']
    df = df[df[0] != 'have no liability for errors in this transcript or for lost profits, losses, or direct, indirect,']
    df = df[df[0] != 'incidental, consequential, special or punitive damages in connection with the']
    df = df[df[0] != 'furnishing, performance or use of such transcript. Neither the information nor any']
    df = df[df[0] != 'opinion expressed in this transcript constitutes a solicitation of the purchase or sale of']
    df = df[df[0] != 'securities or commodities. Any opinion expressed in the transcript does not necessarily']
    # df = df[df[0] != 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights']  
    df = df[df[0] != 'reserved. Any reproduction, redistribution or retransmission is expressly prohibited.']
    # ¬© could not be identified, would apply re
    
    def drop_Bloomberg_mark(x):
        Bloomberg_mark = re.findall(r'reflect the views of Bloomberg LP', x) # 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights'
        if Bloomberg_mark == []:
            return True
        else:
            return False

    true_false = df[0].apply(lambda x: drop_Bloomberg_mark(x))
    df = df[true_false]

    # drop the empthy row
    df = df[df[0] != '']
    df = df[df[0] != '']

    return df

def participants_list(df):
    # reset the index to make sure the index is continuous for better processing
    df = df.reset_index(drop=True)

    #  'Company Participants' index
    # df.loc[df[0] == 'Company Participants']
    Participant_start_index = df.index[df.iloc[:,0] == 'Company Participants'].tolist()
    #  'Other Participants' index
    # df.loc[df[0] == 'Other Participants']
    Participant_middle_index = df.index[df.iloc[:,0] == 'Other Participants'].tolist()
    #  'MANAGEMENT DISCUSSION SECTION' index, is the beginning of the management discussion, would stop before this row
    # df.loc[df[0] == 'MANAGEMENT DISCUSSION SECTION']
    Participant_end_index = df.index[df.iloc[:,0] == 'MANAGEMENT DISCUSSION SECTION' ].tolist()
    # try to find the 'MANAGEMENT DISCUSSION SECTION' or 'Presentation' index
    if Participant_end_index == []:
        Participant_end_index = df.index[df.iloc[:,0] == 'Presentation'].tolist()
        Participant_end_index = [Participant_end_index[-1]]
    # some transcript dont have 'Other Participants'
    if Participant_middle_index == []:
        Participant_middle_index = Participant_end_index

    print(Participant_start_index, Participant_middle_index, Participant_end_index)

    # make the list of company_paticipants and other_participants
    company_paticipants = df.loc[Participant_start_index[0]+1:Participant_middle_index[0]-1]
    company_paticipants.drop(company_paticipants.index[company_paticipants.iloc[:,0] == ''].tolist(), inplace=True)
    company_paticipants = company_paticipants.values.tolist()

    other_paticipants = df.loc[Participant_middle_index[0]+1:Participant_end_index[0]-1]
    other_paticipants.drop(other_paticipants.index[other_paticipants.iloc[:,0] == ''].tolist(), inplace=True)
    other_paticipants = other_paticipants.values.tolist()

    #print("==========================")
    #print("the company paticipants is: ", company_paticipants)
    #print("==========================")
    #print("the other paticipants is: ", other_paticipants)

    #%%
    # after extract the paticipants, we can drop those information to make the transcript more clear
    df = df.reset_index(drop=True)
    df = df.drop(range(df.index[df.iloc[:,0] == 'Company Participants'].tolist()[0],df.index[df.iloc[:,0].isin(['MANAGEMENT DISCUSSION SECTION','Presentation'])].tolist()[0]+1))

    # drop the first row of the df
    df = df.reset_index(drop=True)
    df = df.iloc[1: , :]


    # reset the index again to make sure the index is continuous for better processing
    df = df.reset_index(drop=True)
    # # save to csv
    # df.to_csv('/Users/timliu/Desktop/output/df.csv')
    return df, company_paticipants, other_paticipants

### df_clean_na

In [ ]:
#path = "/Users/hienanh/Documents/GitHub/final_01/test_trans"#Transcript_test#/European (Re)Insurers/HNR1 GY" 
path = "/Users/hienanh/Documents/GitHub/final_01/Transcript_test"
save_path = "/Users/hienanh/Documents/GitHub/final_01/Output"

# all files path
company_paths = []
sectors = os.listdir(path) 
for sector in sectors:
    # path to each sector files
    if sector != '.DS_Store':
        sector_path = path+"/"+sector
    
    # path to each company files
    companies = os.listdir(sector_path) 
    for company in companies:
        if company != '.DS_Store':
            company_path = sector_path+"/"+company
            company_paths.append(company_path)

# create dataframe
df = pd.DataFrame()
df_clean_na = pd.DataFrame(np.zeros((2500,1)), columns=['index']) # create a dataframe with 2500 rows
all_participants = []            

for single_path in company_paths:
    files = os.listdir(single_path)
    for file in files:
        print(file)
        if file.endswith(".pdf"):
            # print(file)
            # Load PDF
            with open(single_path+"/"+file, "rb") as f:
                pdf = pdftotext.PDF(f)
            # Save all text to a txt file.
            with open(save_path+"/"+file.replace(".pdf", ".txt"), "w") as f:
                f.write("\n\n".join(pdf))
            # open the text file
            with open(save_path+"/"+file.replace(".pdf", ".txt")) as f:
                contents = f.readlines()
                df_clean = cleaning_text(contents)
                # extract all the participants
                df_pure_text,company_paticipants,other_paticipants = participants_list(df_clean)
                all_participants.append(company_paticipants)
                all_participants.append(other_paticipants)
                # using the file name to set as the dataframe's column name
                # df[f"{files.index(file)}"] = df_clean
                df[f"{files[files.index(file)]}"] = df_pure_text
                df_clean_na[f"{files[files.index(file)]}"] = df[f"{files[files.index(file)]}"].dropna(inplace=False).reset_index(drop=True)

# drop the first column of the df
df_clean_na = df_clean_na.iloc[:,1:]
df_clean_na.head(5)

20121119_Sompo_Holdings_Inc-_Earnings_Call_2012-11-20_RT000000002067692926.pdf
[1] [4] [12]
20191128_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-11-28_SD000000002903087817.pdf
[2] [8] [15]
20180524_Sompo_Holdings_Inc-_M-A_Call_2018-5-24_FS000000002476688929.pdf
[1] [3] [10]
20190520_Sompo_Holdings_Inc-_Earnings_Call_2019-5-20_SD000000002902929962.pdf
[1] [4] [11]
20120518_Sompo_Holdings_Inc-_Earnings_Call_2012-5-21_RT000000002050742732.pdf
[1] [4] [14]
20141119_Sompo_Holdings_Inc-_Earnings_Call_2014-11-24_FS000000002417374854.pdf
[1] [4] [11]
20151118_Sompo_Holdings_Inc-_Earnings_Call_2015-11-20_DN000000002241138647.pdf
[1] [3] [12]
20131119_Sompo_Holdings_Inc-_Earnings_Call_2013-11-25_DN000000002116348219.pdf
[1] [3] [10]
.DS_Store
20181119_Sompo_Holdings_Inc-_Earnings_Call_2018-11-19_SD000000002905610039.pdf
[1] [4] [12]
20151126_Sompo_Holdings_Inc-_Conf-Presentation_Call_2015-11-26_SD000000002889371132.pdf
[1] [4] [12]
20190528_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-5-2

[1] [4] [9]
20140213_TOKIO_MARINE_HD-_Earnings_Call_2014-2-18_DN000000002126578586.pdf
[1] [4] [12]
20111130_TOKIO_MARINE_HD-_Earnings_Call_2011-12-29_RT000000002039512340.pdf
[1] [7] [12]
20170519_TOKIO_MARINE_HD-_Earnings_Call_2017-5-19_SD000000002893098131.pdf
[1] [4] [11]
20160212_TOKIO_MARINE_HD-_Earnings_Call_2016-2-12_FS000000002253111492.pdf
[1] [4] [9]
.DS_Store
20160809_TOKIO_MARINE_HD-_Earnings_Call_2016-8-9_FS000000002298049534.pdf
[1] [3] [11]
20170808_TOKIO_MARINE_HD-_Earnings_Call_2017-8-8_FS000000002365225099.pdf
[1] [4] [13]
20151118_TOKIO_MARINE_HD-_Earnings_Call_2015-11-18_FS000000002240862350.pdf
[1] [3] [11]


/var/folders/yp/gsl3sklx6bg6x689_qhhd7300000gn/T/ipykernel_32821/433259831.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_clean_na[f"{files[files.index(file)]}"] = df[f"{files[files.index(file)]}"].dropna(inplace=False).reset_index(drop=True)
/var/folders/yp/gsl3sklx6bg6x689_qhhd7300000gn/T/ipykernel_32821/433259831.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{files[files.index(file)]}"] = df_pure_text


20210527_TOKIO_MARINE_HD-_Shareholder_Mtg_Call_2021-5-27_SD000000002958087539.pdf
[1] [9] [18]
20120531_TOKIO_MARINE_HD-_Conf-Presentation_Call_2012-5-31_SD000000002867310091.pdf
[2] [9] [17]
20130213_TOKIO_MARINE_HD-_Earnings_Call_2013-2-13_DN000000002077439881.pdf
[1] [3] [8]
20150520_TOKIO_MARINE_HD-_Earnings_Call_2015-5-20_DN000000002212037068.pdf
[1] [3] [3]
20170519_TOKIO_MARINE_HD-_Earnings_Call_2017-5-19_FS000000002351735615.pdf
[1] [4] [11]
20191003_TOKIO_MARINE_HD-_M-A_Call_2019-10-3_SD000000002888014841.pdf
[2] [4] [4]
20190527_TOKIO_MARINE_HD-_Conf-Presentation_Call_2019-5-27_SD000000002902936847.pdf
[1] [11] [19]
20180214_TOKIO_MARINE_HD-_Earnings_Call_2018-2-14_FS000000002400295131.pdf
[1] [3] [7]
20131119_TOKIO_MARINE_HD-_Earnings_Call_2013-11-20_DN000000002115755239.pdf
[1] [5] [9]
20130809_TOKIO_MARINE_HD-_Earnings_Call_2013-8-12_DN000000002102494109.pdf
[1] [4] [11]
20200520_TOKIO_MARINE_HD-_Earnings_Call_2020-5-20_RT000000002841939564.pdf
[1] [3] [3]
20160520_TOKIO_M

[1] [4] [12]
20150224_QBE_INSURANCE-_Earnings_Call_2015-2-23_FS000000002195934790.pdf
[1] [5] [13]
20190815_QBE_INSURANCE-_Earnings_Call_2019-8-15_SD000000002887415514.pdf
[1] [4] [11]
20110227_QBE_INSURANCE-_Earnings_Call_2011-2-27_SD000000002767279336.pdf
[1] [4] [15]
20181210_QBE_INSURANCE-_Guidance_Call_2018-12-10_SD000000002901842698.pdf
[3] [6] [17]
20210219_QBE_INSURANCE-_Earnings_Call_2021-2-18_DN000000002950935440.pdf
[1] [5] [15]
20180225_QBE_INSURANCE-_Earnings_Call_2018-2-25_SD000000002902635797.pdf
[1] [4] [18]
20150818_QBE_INSURANCE-_Earnings_Call_2015-8-17_FS000000002226588573.pdf
[1] [5] [14]
20121112_QBE_INSURANCE-_Guidance_Call_2012-11-12_SD000000002822570969.pdf
[1] [4] [16]
20180816_QBE_INSURANCE-_Earnings_Call_2018-8-15_DN000000002458573055.pdf
[1] [4] [10]
20180121_QBE_INSURANCE-_Guidance_Call_2018-1-23_FS000000002406211386.pdf
[1] [3] [13]
20170227_QBE_INSURANCE-_Earnings_Call_2017-2-26_FS000000002332481646.pdf
[1] [6] [17]
20190814_QBE_INSURANCE-_Earnings_Call_2

## All participants

In [ ]:
# get the value inside the all_participants 
all_participants = [item for sublist in all_participants for item in sublist]
all_participants = [i[0] for i in all_participants]
# print(all_participants)
# %%
# exclude the title of the participants, i.e.'Roland Vogel, CFO' to 'Roland Vogel" by using re
all_participants = [re.sub(r'\,.*', '', participant) for participant in all_participants]
# exclude the 'Property & Casualty Reinsurance'
all_participants = [re.sub(r'Property & Casualty Reinsurance', '', participant) for participant in all_participants]
# exclude the '[0682QB-E Ulrich Wallin]'
all_participants = [re.sub(r'\[0682QB-E Ulrich Wallin\]', '', participant) for participant in all_participants]
# drop duplicated participants
# all_participants = [i[0] for i in all_participants]
# drop the empty string
all_participants = [participant for participant in all_participants if participant != '']
# remove the sapce in the string
all_participants = [participant.strip() for participant in all_participants]
# add the 'Operator' to the list
all_participants.append('Operator')

# drop the duplicated participants
all_participants_copy = all_participants.copy()
all_participants = []
# drop the duplicated participants
for i in all_participants_copy: 
    if i not in all_participants: 
        all_participants.append(i) 

all_participants = sorted(all_participants)

## new_df

In [ ]:
new_df = pd.DataFrame()
# identify the len before NaN of each column
for column in df_clean_na.columns:
    # end_index = len(df_clean_na[column])-df_clean_na.isnull().sum(axis = 0)[column]-1
    # identify all the rows in df with all_participants in it
    both_participants_row_index = df_clean_na[df_clean_na[column].isin(all_participants)].index.tolist()
    # # append the end_index to the end of both_participants_row_index
    # both_participants_row_index.append(end_index)
    # apply the both_participants_row_index to the df_clean_na['participants']
    new_df[column] = df_clean_na[column]
    new_df[f"participants_{column}"] = df_clean_na[column].apply(lambda x: x if x in all_participants else np.nan)
    # fill the NaN with the value of the previous row
    new_df[f"participants_{column}"] = new_df[f"participants_{column}"].fillna(method='ffill')
    # # exclude the row if pure_df[column]==pure_df[f"participants_{column}"]
    # pure_df = pure_df[pure_df[column] != pure_df[f"participants_{column}"]]
new_df

## pure_df

In [ ]:
pure_df = pd.DataFrame()
# identify the len before NaN of each column
for column in df_clean_na.columns:
    # exclude the row if pure_df[column]==pure_df[f"participants_{column}"]
    pure_df = new_df[new_df[column] != new_df[f"participants_{column}"]]
# drop the column if the column start with participants
pure_df = pure_df.drop(pure_df.columns[pure_df.columns.str.startswith('participants_')], axis=1).T

# append the text of each roll into one string by using s.str.cat(sep='. ')
pure_df = pure_df.apply(lambda x: x.str.cat(sep='. '), axis=1)
# change the pure_df to dataframe
pure_df = pd.DataFrame(pure_df)
# rename the column
pure_df.columns = ['meeting_text']
# extract the index as column from the text
pure_df['file_name'] = pure_df.index
# extract the date from the index column
pure_df['date'] = pure_df['file_name'].apply(lambda x: x.split('_')[0])
# change the date column to datetime
pure_df['date'] = pd.to_datetime(pure_df['date'])
# reset the index
pure_df = pure_df.reset_index(drop=True)
pure_df


## take a sample

In [ ]:
df_sample = pure_df.sample(frac=0.50)

In [ ]:
df_sample